In [15]:
import torch
import time
from IPython.display import clear_output
from time import time

In [12]:
src = [0,1,0,4,4,1,4,3,3,2,2,1,5,3]
dst = [1,0,4,0,1,4,3,4,2,3,1,2,3,5]
edge_weight = [1, 1,1,1,1,1,1,1,1,1,1,1,1,1,1]
A = torch.sparse_coo_tensor(indices=torch.tensor([src, dst]), values=edge_weight, dtype=torch.float32)
r1 = 0
r2 = 1

In [15]:
print(A.to_dense())
print('--------------------------')
print(A)

tensor([[0., 1., 0., 0., 1., 0.],
        [2., 0., 1., 0., 1., 0.],
        [0., 1., 0., 1., 0., 0.],
        [0., 0., 1., 0., 1., 1.],
        [1., 1., 0., 1., 0., 0.],
        [0., 0., 0., 1., 0., 0.]])
--------------------------
tensor(indices=tensor([[0, 1, 1, 0, 4, 4, 1, 4, 3, 3, 2, 2, 1, 5, 3],
                       [1, 0, 0, 4, 0, 1, 4, 3, 4, 2, 3, 1, 2, 3, 5]]),
       values=tensor([1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
                      1.]),
       size=(6, 6), nnz=15, layout=torch.sparse_coo)
tensor(indices=tensor([[0, 0, 1, 1, 1, 2, 2, 3, 3, 3, 4, 4, 4, 5],
                       [1, 4, 0, 2, 4, 1, 3, 2, 4, 5, 0, 1, 3, 3]]),
       values=tensor([1., 1., 2., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.]),
       size=(6, 6), nnz=14, layout=torch.sparse_coo)


In [4]:
# look-up
A[0]

tensor(indices=tensor([[1, 4]]),
       values=tensor([1., 1.]),
       size=(6,), nnz=2, layout=torch.sparse_coo)

In [5]:
# initial node feature
# X = torch.diag_embed(torch.tensor([1., 1., 1., 1., 1., 1.]))
X = (r1 - r2) * torch.rand(size=(6, 3)) + r2
X

tensor([[0.9358, 0.4032, 0.3816],
        [0.9314, 0.4971, 0.1345],
        [0.2599, 0.5998, 0.0767],
        [0.6099, 0.9369, 0.4610],
        [0.0375, 0.5048, 0.9919],
        [0.0321, 0.6819, 0.0410]])

In [6]:
A[0].coalesce().values().unsqueeze(0).T

tensor([[1.],
        [1.]])

In [7]:
# operate Message Passing
X[0] + (
    X[A[0].coalesce().indices()].squeeze() * 
    A[0].coalesce().values().unsqueeze(0).T
    ).sum(dim=0)

tensor([1.9048, 1.4050, 1.5080])

In [8]:
# GCN version
# A_hat = A.to_dense()
A_hat = A.to_dense() + torch.diag_embed(torch.tensor([1., 1., 1., 1., 1., 1.]))
A_hat.to_dense() @ X

tensor([[1.9048, 1.4050, 1.5080],
        [2.1647, 2.0048, 1.5847],
        [1.8012, 2.0337, 0.6722],
        [0.9394, 2.7233, 1.5706],
        [2.5146, 2.3419, 1.9690],
        [0.6420, 1.6187, 0.5020]])

In [9]:
X = (r1 - r2) * torch.rand(size=(6, 2)) + r2
X

tensor([[0.2579, 0.8690],
        [0.9049, 0.2583],
        [0.9995, 0.3938],
        [0.9352, 0.0540],
        [0.5199, 0.7551],
        [0.0026, 0.4244]])

In [10]:
# over-smoothing
temp = X
for i in range(10):
    print(temp)
    temp = torch.sigmoid( A_hat.to_dense() @ temp)
    time.sleep(0.5)
    clear_output(True)


tensor([[0.9482, 0.9482],
        [0.9793, 0.9793],
        [0.9481, 0.9481],
        [0.9774, 0.9774],
        [0.9799, 0.9799],
        [0.8630, 0.8630]])
